<a href="https://colab.research.google.com/github/ljkrajewski/jupyter_notebooks/blob/main/ollama/convert_hfllm_to_gguf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Tutorial: How to convert HuggingFace model to GGUF format](https://github.com/ggml-org/llama.cpp/discussions/2948)

In [ ]:
model_id = "Transducens/empathetic-teacher" #@param string
revision_branch = "main" #@param string
outtype = "q8_0" #@param ["q8_0", "f32", "f16"]
#local_dir_name="vicuna-hf" #@param string

In [ ]:
!pip install huggingface_hub

from huggingface_hub import snapshot_download

#model_id="WhiteRabbitNeo/WhiteRabbitNeo-2.5-Qwen-2.5-Coder-7B"
snapshot_download(
    repo_id=model_id,
    local_dir="vicuna-hf",
    revision=revision_branch,
    etag_timeout=60,
    resume_download=True
)


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
!pip install -r llama.cpp/requirements.txt
#!python llama.cpp/convert_hf_to_gguf.py -h

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py vicuna-hf --outfile "{ model_id.split('/')[1] }-{ outtype }.gguf" --outtype "{ outtype }"
# or "--outtype q16_0" or "--outtype q32_0" to preserve original quality.

In [ ]:
import os
from google.colab import userdata
userdata.get('HF_TOKEN')
os.environ["HUGGING_FACE_HUB_TOKEN"] = userdata.get('HF_TOKEN')

CONTENT=f"""
from huggingface_hub import HfApi
api = HfApi()

model_id = "ljkrajewski/{ model_id.split('/')[1] }-{ outtype }-gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")
api.upload_file(
    path_or_fileobj="{ model_id.split('/')[1] }-{ outtype }.gguf",
    path_in_repo="{ model_id.split('/')[1] }-{ outtype }.gguf",
    repo_id=model_id,
)
"""
with open("upload.py", "w") as f:
  f.write(CONTENT)

!python upload.py